In [ ]:
import os
import time
from typing import List

import seaborn as sns
import matplotlib.pyplot as plt
#import progressbar
from math import pi

#%config Completer.use_jedi = False
import mols2grid
from tqdm import tqdm, tqdm_notebook
from ipywidgets import widgets
from rdkit import Chem, DataStructs
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, Descriptors3D, Draw, rdMolDescriptors, Draw, PandasTools
from rdkit.DataManip.Metric.rdMetricMatrixCalc import GetTanimotoSimMat, GetTanimotoDistMat
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles
from rdkit.Chem import PandasTools
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score,make_scorer,mean_squared_error
import scipy.stats
from sklearn import model_selection

from sklearn.cluster import KMeans
from sklearn import datasets, decomposition
from sklearn.manifold import TSNE
from skopt import BayesSearchCV
PandasTools.RenderImagesInAllDataFrames(images=True)
import umap
import pandas as pd
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from time import  time
from sklearn.cluster import KMeans
IPythonConsole.ipython_useSVG = True 

In [ ]:
import ast
def read_smiles_from_file(path: str) -> List[str]: 
    dict_from_file = []
    with open(path,'r', encoding='utf-8',errors="ignore") as inf:
                for line in inf.readlines():
                        #line = line.replace(" {","{")
                        dict_from_file.append(line)#ast.literal_eval(line))
    return dict_from_file

In [ ]:
def get_largest_fragment_from_smiles(s: str):
    mol = Chem.MolFromSmiles(s)
    if mol:
        clean_mol = LargestFragmentChooser().choose(mol)
        return Chem.MolToSmiles(clean_mol)
    return None

def compute_ecfp_descriptors(smiles_list: List[str]):
    """ Computes ecfp descriptors """
    
    keep_idx = []
    descriptors = []
    for i, smiles in enumerate(smiles_list):
        ecfp = _compute_single_ecfp_descriptor(smiles)
        if ecfp is not None:
            keep_idx.append(i)
            descriptors.append(ecfp)

    return np.vstack(descriptors), keep_idx

def _compute_single_ecfp_descriptor(smiles: str):
    try:
        mol = Chem.MolFromSmiles(smiles)
    except Exception as E:
        return None

    if mol:
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        return np.array(fp)
    
    return None

In [ ]:
smiles = read_smiles_from_file("smiles.txt")
print(smiles[0:10])
efcp,y= compute_ecfp_descriptors(smiles)
params = {"metric" : "jaccard" , "n_neighbors" : 50, "n_components" : 2, "low_memory" :False, "min_dist" : 1e-3}        
#cv = model_selection.check_cv()
#scorer = make_scorer(mean_squared_error)
#b_umap_model = BayesSearchCV(umap.UMAP(),params,n_iter=1,random_state=42,scoring=scorer)

#b_umap_model.fit(efcp,y)
umap_model = umap.UMAP(**params)
X_umap = umap_model.fit_transform(efcp)
UMAP_0, UMAP_1 = X_umap[:,0], X_umap[:,1]
df = pd.DataFrame(data={"SMILES":smiles,"UMAP_0":UMAP_0,"UMAP_1":UMAP_1})
df.head()

In [ ]:
umaps = df[['UMAP_0','UMAP_1']]
params = {"n_clusters":[1,100],"tol":[1e-6,1e-2],"algorithm":["lloyd", "elkan", "auto", "full"]}        

b_kmeans = BayesSearchCV(KMeans(),params,random_state=42,n_iter=1)
b_kmeans.fit(umaps)
b_cluster_labels = b_kmeans.predict(umaps)
print(b_kmeans.best_params_)
silhouette_avg = silhouette_score(umaps,b_cluster_labels)
print("The average silhouette_score is :", silhouette_avg)

In [ ]:
from IPython.display import SVG
import plotly.express as px
kmeans = KMeans(n_clusters=134, random_state=10) # We define the best number of clusters (3)
clusters = kmeans.fit(df[['UMAP_0','UMAP_1']]) #TC1vs TC2

df['Cluster'] = pd.Series(clusters.labels_, index=df.index)
X = df[['UMAP_0', 'UMAP_1', 'Cluster']]

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

# fig = px.scatter_3d(
#     components, x=0, y=1, z=2, color=df['Cluster'],   
#     color_discrete_sequence=['red','green'], 
#     labels={'0': 'UMAP_0'}
# )
# fig = px.scatter(
#     components,x=0,y=1, color=df['pred_y'],
#     color_continuous_scale= ['red','green'],
#     labels={'0': 'UMAP_0', '1': 'UMAP_1', 'color': 'activity'}
# )
#get discordant molecules
# discordants = {d: [] for d in df.SMILES}
# for d in df.index:
#     if df.loc[d,"pred_y"] != df.loc[d,"original_y"]:
#         discordants[df.loc[d,"SMILES"]].append(df.loc[d,"pred_y"])
#         discordants[df.loc[d,"SMILES"]].append(df.loc[d,"original_y"])
#     else:
#         discordants[df.loc[d,"SMILES"]].append(df.loc[d,"pred_y"])
#         discordants[df.loc[d,"SMILES"]].append(df.loc[d,"original_y"])
# discordants = {k: np.mean(v) for k, v in discordants.items()}
#df["discordant"] = df["SMILES"].map(discordants)
fig = px.scatter_3d( components,labels={'y': 'UMAP_0', 'z': 'UMAP_1',"x":"Cluster", 'color': 'activity'},x=df["Cluster"],y=df["UMAP_0"],z=df["UMAP_1"], color_continuous_scale= ['red','blue'])
fig.update_traces(marker_size=5)
fig.show()